<a href="https://colab.research.google.com/github/LangLan-Z/TS/blob/main/Copy_of_MLP3_1_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

#Setting seaborn default theme
sns.set()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Time Series

Time series forecasting is the use of a model to predict future values based on previously observed values. It can broadly be categorized into the following categories:

*   Classical / Statistical Models — Moving Averages, Exponential Smoothing,
ARIMA, SARIMA, TBATS
*   Machine Learning — Linear Regression, XGBoost, Random Forest, or any ML model with reduction methods
* Deep Learning — RNN, LSTM


Time series are widely used for non-stationary data, like economic, weather, stock price, and retail sales (which we will use in this assignment).

## Dataset
We are using Superstore sales data that can be found in the [dataset-folder](https://drive.google.com/drive/folders/1QIgKvGSeltdzH6HMLlEM_ddYYObsrBO2?usp=sharing). Please download the dataset, and upload to your drive.

There are several categories in the Superstore sales data, but we are going to use/predict the furniture sales.

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/datasets/Sample - Superstore.xlsx")
furniture = df.loc[df['Category'] == 'Furniture']

In [ ]:
# Check the date-range
furniture['Order Date'].min(), furniture['Order Date'].max()

We see we have over 4 years of furniture sales data.

In [ ]:
furniture.head()

## Data preprocessing

For the sake of training our first Time Series model, we are only going to use the date-column and the corresponding furniture sales. The other columns we will drop.

In [ ]:
furniture = furniture[['Order Date','Sales']].sort_values('Order Date')

In [ ]:
# Check for missing values
furniture.isnull().sum()

In [ ]:
furniture.head(10)

We see we can have several sales on one date. We are only interested in the total sales on a given day, so we sum these values.

In [ ]:
furniture = furniture.groupby('Order Date')['Sales'].sum().reset_index()

Set the index to the date column:

In [ ]:
furniture = furniture.set_index('Order Date')
furniture.index

## Visualising

Now that we have selected the right data and made sure it is in the right format, let's take a look at it by visualising it in a Matplotlib-plot.

In [ ]:
furniture.plot(figsize=(15, 6))
plt.show()

This looks complicated, since there are a lot of values, and no clear trend or seasonality.

Instead of looking at the sales of each individual day, let's model the average daily sales of a month (using the start of each month as the timestamp).

In [ ]:
data = furniture['Sales'].resample('MS').mean()

In [ ]:
# Plot result
data.plot(figsize=(15, 6))
plt.show()

This already looks a lot cleaner, better suited for our first time creating a Time Series model.

When looking at the plot, we can already see some patterns. Such as sales are always low at the beginning of the year and higher at the end of the year.

We can also visualize our data using a method called time-series decomposition that allows us to decompose our time series into three distinct components: trend, seasonality, and noise.

In [ ]:
from pylab import rcParams
import statsmodels.api as sm

rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.seasonal_decompose(data, model='additive')
fig = decomposition.plot()
plt.show()

We indeed see some seasonality patterns here, but no clear trend.

## Modeling - ARIMA

We are going to apply one of the most commonly used method for time-series forecasting, known as ARIMA, which stands for Autoregressive Integrated Moving Average.

Simplified, the different parts of the model do this:
* Auto Regressive (AR) models are a specific type of regression model where, the dependent variable depends on past values of itself
* Integrated (I)(though it has little to do with integration). It just means that, instead of predicting the time series itself we will predict the differences of the series from one time step to the next time step.
* Moving Average (MA) models work by analysing how wrong you were in predicting values for the previous time-periods to make a better estimate for the current time-period.

These three models work together to create the best possible model.

They come together in ARIMA(p, d, q) where,
* p = number of prior lag observations we include in the model (from AR part)
* d = order of differencing, i.e. a single phase of differencing (from I part)
* q = size of moving average window (from MA part)

### Training

We are going to set these parameters and train our furniture’s sales ARIMA Time Series Model.

An additional parameter we will use is M, which brings a seasonal component to the model.
M indicates the periodicity, i.e. the number of periods in season, such as 12 for monthly data (which is also a good number for our data, which has a clear pattern each year).

Let's take an order of 1 for the first three parameters: include 1 prior observation, difference over 1 prior observation and have a moving average window of 1 observation.
For the seasonal order we will use the same parameters, only no moving average window, and use an M of 12.

(NB: An ideal way of setting these parameters would be to do a grid-search.)

In [ ]:
# Initialize the model
mod = sm.tsa.statespace.SARIMAX(data,
                                order=(1, 1, 1), #(p,d,q)
                                seasonal_order=(1, 1, 0, 12)) #(p,d,q,M)

In [ ]:
# Train the model
results = mod.fit()

## Testing

To help us understand the accuracy of our forecasts, we compare predicted sales to the real sales of the time series. We set forecasts to start at 2017–01–01.

In [ ]:
# Predict from 2017
pred = results.get_prediction(start=pd.to_datetime('2017-01-01'), dynamic=False)
pred_ci = pred.conf_int() #confidence interval

# Create result-plot
ax = data['2014':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()

The line plot is showing the observed values compared to the rolling forecast predictions. Overall, our forecasts align with the true values quite well, showing the lower sales at the start of the year, and the higher sales at the end of the year.

## Predicting

With our trained model, we can now predict beyond the scope of our data. This is easily done, using the .get_forecast() function, giving it the number of steps to forecast as a parameter.

In [ ]:
# Make predictions for next 100 steps
pred = results.get_forecast(steps=100)
pred_ci = pred.conf_int() #confidence interval

In [ ]:
# Plot the predictions
ax = data.plot(label='observed', figsize=(14, 7))
pred.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()

We clearly see the furniture sales seasonality. And as we get further into the future, the model becomes less confident of it's predictions (see the grey area of the confidence intervals).

## Assignment 3.1

**Question 1**

Now try it yourself, using the same dataset, but selecting another sales category, like 'Office Supplies'.

*   Check the data for missing values, and aggregate if necessary
*   Explore the data
*   You might also want to use daily average sales
*   Train and test your model
*   Predict future values and visualize the results

